# 要約 
このJupyterノートブックは、LMSYS - Chatbot Arena コンペティションにおける人間の好み予測モデルの構築を目的としています。具体的には、異なる大規模言語モデル（LLM）の応答がどちらがユーザーに好まれるかを予測するための強化学習を活用した選好モデルのファインチューニングと推論を行っています。

### 主な内容と問題設定
- **問題**: モデルは、ユーザーの選好を予測するために、異なるLLMが生成した応答を受け取り、どちらの応答がより好まれるか（または同点）を予測することを目的としています。具体的には、ユーザーから提供されたプロンプトに対する応答Aと応答Bを比較します。
  
### 使用する手法やライブラリ
1. **ライブラリのインストール**:
   - `bitsandbytes`, `transformers`, `tokenizers`, `peft` などの必須ライブラリをインストールしています。

2. **データ準備**:
   - Kaggleからテストデータを読み込み、プロンプトと応答をトークン化する関数`tokenize`を定義し、データセットを適切な長さに調整しています。最大シーケンス長は2400に設定。

3. **モデルの構築**:
   - `Llama3ForSFT`クラスを定義し、事前訓練されたLLaMAモデルをベースとしたモデルを構築しています。これにより、入力データを受け取り、隠れ状態からロジットを計算します。

4. **推論**:
   - 複数のGPUを活用して推論を行い、各応答（A, B）の勝率を計算します。また、結果をDataFrameに格納し、最終的な提出ファイル（submission.csv）を作成しています。

5. **マルチスレッド処理**:
   - `ThreadPoolExecutor`を使用して、効率的に推論を実行し、計算時間を短縮しています。

### 結果の提出
- モデルの推論結果は`submission.csv`として保存され、各IDに対する応答の勝率（winner_model_a, winner_model_b, winner_tie）を含む形式で生成されています。

このノートブックは、現実の人間の選好を機械学習技術を用いて予測するための基盤として、特にLLMのトレーニングと評価に焦点を当てています。

---


# 用語概説 
以下は、初心者が『LMSYS - Chatbot Arena 人間による好み予測チャレンジ』のノートブックを読んでいて、理解に苦しむ可能性がある専門用語の解説です。

1. **PEFT（Parameter-Efficient Fine-Tuning）**:
   - モデルの全てのパラメータを調整することなく、一部のパラメータのみを調整することで、モデルのパフォーマンスを向上させる技術です。この技術は、特に大規模なモデルを効率的にファインチューニングする際に有効です。

2. **LoRA（Low-Rank Adaptation）**:
   - PGFTと同様に、既存の重みを固定し、補助的な低ランクの重みを追加することでモデルの適応を行う手法です。これにより、少ないパラメータで効率的にファインチューニングが可能となります。

3. **Causal Language Model (CausalLM)**:
   - 自己回帰型言語モデルで、シーケンスの次の単語を予測するために、これまでの単語の情報のみを利用します。例えば、文の最初から最後までの単語を順に生成するモデルです。

4. **ノイズがないトークナイザー**:
   - 通常のトークナイザーは、テキストに対して特殊なトークン（例えば、[CLS]や[SEP]など）を追加しますが、ノイズのないトークナイザーは追加トークンなしで純粋にトークン化のみを行います。文脈の理解に影響を与えないことが期待されます。

5. **アテンションマスク**:
   - トランスフォーマモデルにおいて、入力の一部を無視するためのマスクです。通常、パディングされた部分を無視させるために使われ、アテンション演算を効率化します。

6. **エクストラ・プロンプト**:
   - モデルに与える追加の情報や指示です。通常のプロンプトではカバーできない詳細な指示を提供することで、モデルの出力の品質を向上させることが可能です。

7. **torch.cuda.amp**:
   - 自動混合精度（Automatic Mixed Precision）のこと。計算を行う際に、必要に応じて32ビットと16ビットの浮動小数点を使うことにより、メモリ使用量を削減し、計算速度を向上させる技術です。

8. **モデル並列（Model Parallelism）**:
   - モデルが大きすぎて1つのGPUに収まらない場合、異なる部分のモデルを複数のGPUにわけて実行する手法です。

9. **ロジット（Logits）**:
   - モデルの出力で、クラスごとのスコアや値です。通常、カテゴリ分け問題においてソフトマックス関数を通して各クラスの確率に変換されます。

10. **グラディエント計算**:
    - ネットワークの出力に対する誤差の寄与度を計算し、パラメータの更新に利用するプロセスです。これによりモデルの学習が行われます。

11. **重みのキー（Weight Keys）**:
    - モデルの重みにアクセスする際の識別子。特定の重みを指定し、それらを一括で更新したり参照するために用いられます。

12. **特定のトークン**:
    - モデルによって定義された特別なトークン。例えば、ラベルのトークンIDは特定の応答を示すために使用されることが多いです。

これらの用語についての理解が 進むことで、Jupyter Notebookの内容がより明確に理解できるようになるでしょう。

---


# 注意
- [トレーニングスクリプト](https://www.kaggle.com/code/shelterw/training-llama3-8b-4-bit-qlora-sft)



In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links /kaggle/input/llm-pip-2024727
!pip install -q -U transformers --no-index --find-links /kaggle/input/llm-pip-2024727
!pip install -q -U tokenizers --no-index --find-links /kaggle/input/llm-pip-2024727
!pip install -q -U peft --no-index --find-links /kaggle/input/llm-pip-2024727

In [ ]:
import time
import torch
import sklearn
import numpy as np
import pandas as pd
import torch.nn as nn
from torch.cuda.amp import autocast
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
from threading import Thread
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType 
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import BitsAndBytesConfig, LlamaForCausalLM, LlamaModel, LlamaPreTrainedModel
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from transformers import set_seed

# CUDAのメモリ効率の良いSDPを有効化します
torch.backends.cuda.enable_mem_efficient_sdp(True)
# CUDAのフラッシュSDPを有効化します
torch.backends.cuda.enable_flash_sdp(True)

# GPUデバイスが2つ存在することを確認します
assert torch.cuda.device_count() == 2, "すみません - マルチGPUが必要です！"

In [ ]:
MODEL_NAME = '/kaggle/input/llama-3-1-8b-instruct-bnb-4bit'  # 使用するモデルのパス
WEIGHTS_PATH = '/kaggle/input/sft-llama3-1-lora-9174'  # 学習済みモデルの重みのパス
MAX_LENGTH = 2400  # 最大シーケンス長
BATCH_SIZE = 2  # バッチサイズ
DEVICE = torch.device("cuda")  # CUDAデバイスを使用する

# データの準備



In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込みます

# トークナイザーでトークン化する関数
def tokenize(example, tokenizer):
    prompts = tokenizer(eval(example['prompt'], {"null": ""}), add_special_tokens=False)["input_ids"]  # プロンプトのトークン化
    responses_a = tokenizer(eval(example['response_a'], {"null": ""}), add_special_tokens=False)["input_ids"]  # 応答Aのトークン化
    responses_b = tokenizer(eval(example['response_b'], {"null": ""}), add_special_tokens=False)["input_ids"]  # 応答Bのトークン化
    
    # プロンプト、応答A、応答Bの長さが一致することを確認
    assert len(prompts) == len(responses_a) == len(responses_b), "プロンプト、応答A、応答Bの長さが一致しません"
    
    # プロンプト、応答を逆順にします
    prompts, responses_a, responses_b = prompts[::-1], responses_a[::-1], responses_b[::-1]
    
    prompt, response_a, response_b = [], [], []
    p_len, a_len, b_len = 0, 0, 0
    
    # それぞれのプロンプトと応答の長さを計算します
    for p, a, b in zip(prompts, responses_a, responses_b):
        prompt.append(p)
        response_a.append(a)
        response_b.append(b)
        p_len += len(p)
        a_len += len(a)
        b_len += len(b)
        # 最大シーケンス長を超える場合は停止します
        if p_len + a_len + b_len > MAX_LENGTH:
            break

    # リストをフラット化して逆順に戻します
    prompt = [item for sublist in reversed(prompt) for item in sublist]
    response_a = [item for sublist in reversed(response_a) for item in sublist]
    response_b = [item for sublist in reversed(response_b) for item in sublist]
    
    # トークン数が最大長を超える場合はカットします
    p_a_b_len = len(prompt) + len(response_a) + len(response_b)
    cut_len = p_a_b_len - MAX_LENGTH
    if cut_len > 0:
        prompt = prompt[:-int(len(prompt)/p_a_b_len*cut_len)]
        response_a = response_a[:-int(len(response_a)/p_a_b_len*cut_len)]
        response_b = response_b[:-int(len(response_b)/p_a_b_len*cut_len)]
    
    # トークンを追加します
    prompt = tokenizer('<prompt>: ', add_special_tokens=False)["input_ids"] + prompt
    response_a = tokenizer('\n\n<response_a>: ', add_special_tokens=False)["input_ids"] + response_a
    response_b = tokenizer('\n\n<response_b>: ', add_special_tokens=False)["input_ids"] + response_b
    extra_prompt = tokenizer('\n\n---------\nどちらがプロンプトに対してより良い応答ですか？ a または b または tie ?\n\n回答: ', add_special_tokens=False)["input_ids"]
    label_token_id = [128250]  # ラベル用トークンID
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + label_token_id + [tokenizer.eos_token_id]  # 入力IDを構築
    attention_mask = len(input_ids)*[1]  # アテンションマスクを構築
    labels = [-100]* len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + label_token_id + [tokenizer.eos_token_id]  # ラベルを構築
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# トークン化



In [ ]:
%%time
tokenizer = AutoTokenizer.from_pretrained(WEIGHTS_PATH)  # トークナイザーを読み込みます
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]  # ラベルのトークンIDを取得します

# データを読み込む関数
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)  # パンダのデータフレームからデータセットを作成します
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        # remove_columns=raw_datasets.column_names,  # 列を削除するオプション（コメントアウトされています）
        fn_kwargs={'tokenizer': tokenizer},
    )
    return tokenized_datasets

test_ds = load_data(test, tokenizer)  # データをトークン化します
test_ds

In [ ]:
data = test_ds.to_pandas()  # データセットをパンダのデータフレームに変換します
data["max_len"] = data["input_ids"].apply(len)  # 各入力の最大長を計算します
data[:3]  # データの最初の3行を表示します

In [ ]:
data['input_ids'][0]  # 最初の入力IDを表示します

In [ ]:
print(tokenizer.decode(data["input_ids"][0]))  # 最初の入力IDをデコードして表示します

# モデルの読み込み 
各GPUに1つのモデルを読み込みます。



In [ ]:
class Llama3ForSFT(LlamaPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]  # モデルに関連する重みのキー
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)  # Llamaモデルを初期化
        self.vocab_size = config.vocab_size  # ボキャブラリのサイズを取得
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)  # ロジットを計算するための線形層
        self.post_init()  # モデルの初期化後の処理

    def forward(
        self,
        input_ids= None,
        attention_mask= None,
        position_ids = None,
        past_key_values= None,
        inputs_embeds= None,
        labels= None,
        use_cache= None,
        output_attentions= None,
        output_hidden_states = None,
        return_dict= None,
        cache_position = None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]  # 隠れ状態を取得
        if self.config.pretraining_tp > 1:  # モデル並列が必要な場合
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)  # 重みを分割
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]  # ロジットを計算
            logits = torch.cat(logits, dim=-1)  # 結果を結合
        else:
            logits = self.lm_head(hidden_states)  # 通常のロジット計算

        logits = logits.float()  # ロジットをfloat型に変換

        loss = None
        if labels is not None:  # ラベルがある場合
            # トークンをシフトしてnを予測する
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # トークンをフラット化
            loss_fct = nn.CrossEntropyLoss()  # クロスエントロピーロスを定義
            shift_logits = shift_logits.view(-1, self.config.vocab_size)  # ロジットをフラット化
            shift_labels = shift_labels.view(-1)  # ラベルをフラット化
            # モデル並列を有効にする
            shift_labels = shift_labels.to(shift_logits.device)

            fake_label_tokens_ids = torch.tensor([128250], device=shift_labels.device)  # 偽ラベルのトークンID
            label_tokens_ids = torch.tensor(LABEL_IDS, device=shift_labels.device)  # ラベルのトークンID
            
            # true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]  # 本物のラベルの取得（コメントアウトされています）
            true_logits = shift_logits[torch.isin(shift_labels, fake_label_tokens_ids)][:, label_tokens_ids]  # 本物のロジットを取得

        return CausalLMOutputWithPast(  # 結果を返します
            loss=loss,
            logits=true_logits,
        )

In [ ]:
# GPU 0にベースモデルを読み込みます
device0 = torch.device('cuda:0')
base_model_0 = Llama3ForSFT.from_pretrained(
    MODEL_NAME,
    use_cache=False,
    device_map='cuda:0',
)
# GPU 1にベースモデルを読み込みます
device1 = torch.device('cuda:1')
base_model_1 = Llama3ForSFT.from_pretrained(
    MODEL_NAME,
    use_cache=False,
    device_map='cuda:1',
)

# 重みの読み込み 



In [ ]:
# PEFTを取得します
model_0 = PeftModel.from_pretrained(base_model_0, model_id=WEIGHTS_PATH).to(device0)  # GPU 0にモデルを移動します
model_0.eval()  # 評価モードに設定

model_1 = PeftModel.from_pretrained(base_model_1, model_id=WEIGHTS_PATH).to(device1)  # GPU 1にモデルを移動します
model_1.eval()  # 評価モードに設定

# 推論



In [ ]:
@torch.no_grad()  # 勾配計算を無効にします
@torch.cuda.amp.autocast()  # 自動混合精度を有効にします
def inference(df, model, device, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):
    a_win, b_win, tie = [], [], []  # 各応答の勝率を格納するリスト

    model.eval()  # 評価モードに設定
    for start_idx in range(0, len(df), batch_size):  # バッチ単位でループ
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックス
        tmp = df.iloc[start_idx:end_idx]  # 一時データフレームを作成
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストとして取得
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストとして取得
        labels = tmp["labels"].to_list()  # ラベルをリストとして取得
        
        # パディングを行い、テンソルに変換します
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        input_ids = inputs["input_ids"].to(device)  # デバイスに移動
        attention_mask = inputs["attention_mask"].to(device)  # デバイスに移動
        
        pad_labels = []  # パディングラベルのリストを初期化
        for label in labels:
            # ラベルを入力の長さに合わせてパディングします
            label = list(label) + [tokenizer.pad_token_id]*(input_ids[0].shape[0] - label.shape[0])
            pad_labels.append(label)
        labels = torch.tensor(pad_labels).to(device)  # デバイスに移動
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)  # モデルに入力を渡して出力を取得
        proba = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # ソフトマックスを適用して確率を取得
        
        # 各応答の勝率をリストに追加します
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df['winner_model_a'] = a_win  # 応答Aの勝率
    df['winner_model_b'] = b_win  # 応答Bの勝率
    df['winner_tie'] = tie  # 同点の勝率
    return df  # 結果を返します

In [ ]:
st = time.time()  # 時間計測の開始

# データを最大長でソートします
data = data.sort_values("max_len", ascending=False)
sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータを取得
sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータを取得

# マルチスレッドで推論を実行します
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device0, device1))

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 確率を取得

print(f"経過時間: {time.time() - st}")  # 経過時間を表示

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝率を結果データフレームに保存
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を結果データフレームに保存
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 同点の勝率を結果データフレームに保存
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用データフレームを作成
submission_df.to_csv('submission.csv', index=False)  # CSVファイルとして保存
display(submission_df)  # 提出データフレームを表示

---

# コメント 

> ## Songling
> 
> このファイルが見つかりません。誰か助けてくれませんか？
> 
> 警告: Location '../input/llm-pip-2024-7-4/' は無視されます: 存在しないパスか、特定のスキームが欠如しています。
> 
> エラー: 要件bitsandbytesを満たすバージョンが見つかりません（バージョン: なし）
> 
> エラー: peftに対する一致する配布が見つかりません
> 
> 警告: Location '../input/llm-pip-2024-7-4/' は無視されます: 存在しないパスか、特定のスキームが欠如しています。
> 
> 警告: Location '../input/llm-pip-2024-7-4/' は無視されます: 存在しないパスか、特定のスキームが欠如しています。
> 
> 警告: Location '../input/llm-pip-2024-7-4/' は無視されます: 存在しないパスか、特定のスキームが欠如しています。
> 
> エラー: 要件peftを満たすバージョンが見つかりません（バージョン: なし）
> 
> エラー: 一致する配布が見つかりませんでした

---

> ## Shimei
> 
> こんにちは、この素晴らしいノートブックを共有してくれてありがとう！
> 
> しかし、トレーニングノートブックのファインチューニングの後、推論時に次のエラーが発生します。
> 
> OSError: /kaggle/input/llama3-sft/checkpoint-2800 に config.jsonというファイルが見当たりません。'https://huggingface.co//kaggle/input/llama3-sft/checkpoint-2800/tree/None' を確認して、利用可能なファイルを確認してください。

> ## Romanov_Alex
> > 同じです。修正方法を試してみましたか？

> ## duncangao
> > 私も同じです。修正方法を知りたいです。

---

> ## Huang Jing Stark
> 
> MAX_LENGTHを2400に設定した具体的な理由はありますか？

> ## ShelterWTopic Author
> > モデルにより多くの情報を与えるためです。

---

> ## Rabbit
> 
> モデルをHuggingFaceからKaggleにどうやってロードしますか？

> ## ShelterWTopic Author
> > 新しいノートブックを作成し、インターネットを開き、スナップショットダウンロードを使用してモデルをダウンロードし、このノートブックを新しいデータセットとして使用します。

---

> ## Dlond Mike
> 
> でも、君は本当に英雄だ。

---

> ## PaulRRR
> 
> llama3.1を使用していますが、次のエラーが出ます: ValueError: rope_scalingは二つのフィールド、typeとfactorを持つ辞書である必要があります。{'factor': 8.0, 'high_freq_factor': 4.0, 'low_freq_factor': 1.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}を取得しました。

> ## zhudong1949
> > transformersのバージョンを更新してください。

---

> ## Aaryan Gupta
> 
> こんにちは、このノートブックのデータセットはプライベートですか？コードがパスが見つからないエラーを出しています。

---

> ## 박민욱peterminpark
> 
> 待ってください。
> 
> モデル /kaggle/input/sft-llama3-lora-9231 は公開されていますか？
> 
> また、トレーニングコードを使用してFold0-Fold4モデルを取得し、テストしましたが、0.94 - 0.95範囲で止まっています。

> ## PaulRRR
> > こんにちは、この推論コードを実行できますか？

---

> ## Lorry Zou
> 
> Huggingfaceからリポジトリ全体をダウンロードし、推論用のデータセットを作成しましたが、読み込み時に次のエラーが出ました:
> 
> Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index または flax_model.msgpackが、ディレクトリ/kaggle/input/llama-3-8b-instruct-bnb-4bit-1に見つかりませんでした。 
> 誰か同じ問題に遭遇した人はいますか？解決方法を知っている人はいますか？

> ## Lorry Zou
> > 今、別のエラーが発生しました:
> > 不正な path_or_model_id: '/kaggle/input/llama-3-8b-instruct-bnb-4bit'。ローカルフォルダへのパスまたはHub上のモデルのrepo_idを指定してください。 
> > 誰か助けてください。

---

> ## OHIRA
> 
> 素晴らしい作業に感謝します!!!
> 
> もし8bit LoRAを使用する場合、時間制限内に推論できますか？
> 
> どのくらいの時間ですか？
> 
> 知っているなら教えてください！

> ## ShelterWTopic Author
> > 4ビットでも同様です。

> ## hn
> > こんにちは、gemma2-9bを使おうとしましたが、何故か8ビットを使用し、合計の長さが[256+640+640] = [1536]以上になると常に例外が発生します。以前に経験したことがありますか？4ビットに減らしたら、[256+512+512]で推論が実行されました。

---

> ## Qihang Wang
> 
> こんにちは、label_token_id = [128250] に設定した理由をお聞きしてもいいですか？

> ## ShelterWTopic Author
> > [128250]はllama3の特殊トークンで、意味はありません。ただラベル位置を取得し、他の通常トークンを避けます。

---

> ## Dlond Mike
> 
> こんにちは、この新しいノートブックの新しい点を簡単に説明してもらえますか？

---

> ## YEI0907
> 
> 推論時間はどのくらいかかりましたか？自回帰アーキテクチャを使用しましたが、推論時間が大幅に増加しました。

---

> ## YEI0907
> 
> 老哥，你的这个架构推理时间花了多久呀？我也用了自回归模型进行推理，但是推理时间比直接分类长了很多

> ## ShelterWTopic Author
> > max_len=2400 -> 6h -> 0.935
> > max_len=1024 -> 3h -> 0.938

---

> ## Rabbit
> 
> あなたの方法を試しましたが、cv0.927だけどlbは0.960です。

> ## ShelterWTopic Author
> > 悲しい、過剰適合かもしれませんか？

> ## PaulRRR
> > こんにちは、この推論コードを実行できますか？

> ## Rabbit
> > コードをファインチューニングしました。

---

> ## lllleeeo
> 
> これは本当に素晴らしい作業です！分類ヘッドとのスコアを比較しましたか？

> ## ShelterWTopic Author
> > もちろん、この方法はほぼ同じパラメータでより良く機能します。

---

> ## Octavio Grau
> 
> このノートブックは私にエラーを出します（0.935バージョン）。誰かヒントがありますか？ [@shelterw](https://www.kaggle.com/shelterw) 公開してくれてありがとう！

> ## ShelterWTopic Author
> > どんなエラーですか？

> ## Songling
> > [@shelterw](https://www.kaggle.com/shelterw) 
> > 警告: Location '../input/llm-pip-2024-7-4/' は無視されます: 存在しないパスか、特定のスキームが欠如しています。
> > エラー: 要件bitsandbytesを満たすバージョンが見つかりません（バージョン: なし）
> > エラー: 要件peftを満たすバージョンが見つかりません（バージョン: なし）

> ## ShelterWTopic Author
> > 申し訳ありません、更新しました。

---

> ## Nikhil Tumbde
> 
> ノートブックをありがとう！私はLLMのunslothバージョンを実装しようとしていましたが、できませんでした。これは本当に助かります。 
> スコア計算にどのくらい時間がかかりますか？

> ## ShelterWTopic Author
> > 約6時間です。
> > max_len = 1024に設定すると、3時間で0.938のスコアになります。

> ## Nikhil Tumbde
> > ありがとう！
> > 最後の質問をしてもいいですか（オープンエンドかもしれません、申し訳ありません）。私はNLP分野の初心者で、unslothからシーケンシャル分類への変換中に多くのエラーに直面しました。各関数の期待される出力と入力を知る方法はありますか？chatgptを試しましたが、あまり役立ちませんでした。例えば、
> > Llama3ForSFTはCausalLMOutputWithPastを返す必要があります。
> > loss=loss,
> > logits=true_logits,
> > のように。

> > トークナイズの方は、次のように返す必要があります
> > {
> > "input_ids": input_ids,
> > "attention_mask": attention_mask,
> > "labels": labels
> > }
> > ご時間をいただきありがとうございます！